<a href="https://colab.research.google.com/github/Frexander/Final_Portfolio/blob/master/PSO_VRPTW_alt3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necesarry packages
! pip install pandas
! pip install requests
! pip install numpy

In [2]:
# Import necesarry libraries
import pandas as pd
import requests
import io
import numpy as np
import random
import copy
import time

In [3]:
# Downolad Solomon dataset c101.txt, convert it to a pandas dataset and store it as a csv-file. Save both original .txt- and .csv-file.

def download_solomon_dataset():
  url = "https://people.idsia.ch/~luca/macs-vrptw/problems/c101.txt"

  try:
    response = requests.get(url)          # Downolad dataset
    response.raise_for_status()            # Check if downolad is successful
    content = response.text                # Read dataset and split dataset into lines
    lines = content.strip().split('\n')
    data_lines = lines[9:]                # Exclude the first 9 lines of the dataset (Information about the dataset)
    customer_data = []

    for line in data_lines:               # Split each line into collumns and create a pandas dataset
      values = line.strip().split()
      if len(values) == 7:
        customer_data.append({
            'CUST_NO': int(values[0]),
            'X_COORD': float(values[1]),
            'Y_COORD': float(values[2]),
            'DEMAND': float(values[3]),
            'READY_TIME': float(values[4]),
            'DUE_DATE': float(values[5]),
            'SERVICE_TIME': float(values[6])
        })

    df = pd.DataFrame(customer_data)

    df.to_csv('c101.csv', index = False)
    with open('c101.txt', 'w') as f:
      f.write(content)

    print("Both .csv and .txt files saved successfully")
    return df

  except requests.exceptions.RequestException as e:
    print(f"Download not sucessfull for file: {e}")
    return None

df = download_solomon_dataset()

if df is not None:
  print("First five rows of dataset: ")
  print(df.head(5).to_string(index = False))

Both .csv and .txt files saved successfully
First five rows of dataset: 
 CUST_NO  X_COORD  Y_COORD  DEMAND  READY_TIME  DUE_DATE  SERVICE_TIME
       0     40.0     50.0     0.0         0.0    1236.0           0.0
       1     45.0     68.0    10.0       912.0     967.0          90.0
       2     45.0     70.0    30.0       825.0     870.0          90.0
       3     42.0     66.0    10.0        65.0     146.0          90.0
       4     42.0     68.0    10.0       727.0     782.0          90.0


In [4]:
# Calculate Eucledian distance between each customer and all other customers
# Add a collumn for each customer such that each row (representing one customer) has one cell with distance to each other customer (represented in collumns)
# Visualize first 10 rows

def create_distance_matrix(df):
  customer_details = df[['CUST_NO', 'DEMAND', 'READY_TIME', 'DUE_DATE', 'SERVICE_TIME']].copy()

  distance_columns = {}     # Dictionary for storing all distance columns.

  for dest_customer in df['CUST_NO']:
    x2 = df[df['CUST_NO'] == dest_customer]['X_COORD'].values[0]
    y2 = df[df['CUST_NO'] == dest_customer]['Y_COORD'].values[0]

    distance = []
    for index, row in df.iterrows():
      x1 = row['X_COORD']
      y1 = row['Y_COORD']
      dist = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
      distance.append(round(dist, 2))

    distance_columns[f"DIST_TO_{dest_customer}"] = distance     # Store ech distance column in this dictionary

  distance_df = pd.DataFrame(distance_columns)

  distance_matrix = pd.concat([customer_details, distance_df], axis = 1)

  return distance_matrix

distance_matrix = create_distance_matrix(df)

print("Distance matrix (first 10 rows): ")
print(distance_matrix.head(10).to_string(index = False))

distance_matrix.to_csv('C101 distance_matrix.csv', index = False)

Distance matrix (first 10 rows): 
 CUST_NO  DEMAND  READY_TIME  DUE_DATE  SERVICE_TIME  DIST_TO_0  DIST_TO_1  DIST_TO_2  DIST_TO_3  DIST_TO_4  DIST_TO_5  DIST_TO_6  DIST_TO_7  DIST_TO_8  DIST_TO_9  DIST_TO_10  DIST_TO_11  DIST_TO_12  DIST_TO_13  DIST_TO_14  DIST_TO_15  DIST_TO_16  DIST_TO_17  DIST_TO_18  DIST_TO_19  DIST_TO_20  DIST_TO_21  DIST_TO_22  DIST_TO_23  DIST_TO_24  DIST_TO_25  DIST_TO_26  DIST_TO_27  DIST_TO_28  DIST_TO_29  DIST_TO_30  DIST_TO_31  DIST_TO_32  DIST_TO_33  DIST_TO_34  DIST_TO_35  DIST_TO_36  DIST_TO_37  DIST_TO_38  DIST_TO_39  DIST_TO_40  DIST_TO_41  DIST_TO_42  DIST_TO_43  DIST_TO_44  DIST_TO_45  DIST_TO_46  DIST_TO_47  DIST_TO_48  DIST_TO_49  DIST_TO_50  DIST_TO_51  DIST_TO_52  DIST_TO_53  DIST_TO_54  DIST_TO_55  DIST_TO_56  DIST_TO_57  DIST_TO_58  DIST_TO_59  DIST_TO_60  DIST_TO_61  DIST_TO_62  DIST_TO_63  DIST_TO_64  DIST_TO_65  DIST_TO_66  DIST_TO_67  DIST_TO_68  DIST_TO_69  DIST_TO_70  DIST_TO_71  DIST_TO_72  DIST_TO_73  DIST_TO_74  DIST_TO_75  DIST_TO_76

In [ ]:
# ALTERNATIVE 1: GLOBAL BEST

# Define number of particles, c1, c2, Vmax (negihborhoos size: fully connected swarm)
num_particles = 30
c1 = 2.0
c2 = 2.0
w = 0.7
Vmax = 0.1    # Best guess

# Constraints defined in dataset
vehicle_capacity = 200
max_route_duration = 1236

# Define fitness function / constraints
    # Number of vehicles
    # Distance from depot, through all customers and back to depot
    # Load violation
    # Arriving too early
    # Arriving too late
    # Travel time = dsitance + time difference if arriving too early + service time
    # Cost = distance, time, load violation (10 irrespective of weight overlaod), arriving too late (number of minutes?)

In [ ]:
# Initialise Position (Random routes for each particle, each particle is a nested list of customers in random order of delivery)

# Set personalbest to current position and global best to None


In [ ]:
# Initialise velocity (number of changes in routes per iteration)


In [ ]:
# Update velocity (changes the order of x number of customers in/between y number of routes)


In [ ]:
# Update position (new particles position defined by number of routes and order of customers in each route)

In [ ]:
# The PSO-algorithm

# Define termintaion criteria (number of iterations to start with)

# Random initialisation of position

# Set velocity to 0

    # Evaluate fitness
        # Distance from depot, through all customers and back to depot
        # Load violation
        # Arriving too early
        # Arriving too late
        # Travel time = dsitance + time difference if arriving too early + service time
        # Cost = dsitance, time, load, arriving too late

    # Print cost for each particle, personal best and global best

    # Define velocity

    # Calculate new position

# Visualise progress for each påarticle, improvement in global best and difference between best and worst particle at termination



In [ ]:
# Improvement posibilities
# Include X/Y coordinates for customers
# Group custoers by y-coordinate interval
# Group customers in siimilar y-coordinate intervall based on x-coordinate
# Send separate vehocles to each "neighbourhood" of customers
# Size of neighbourhood is number of customers and depends on level of clustering
# Large neighbourhoods/many customer in same cluster requires mutliple vehicles.